<a href="https://colab.research.google.com/github/lazysimon/NYCDSA_CapstoneProject/blob/master/movies_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import time

# data science imports
import math
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from urllib.request import urlopen

# utils import
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

# visualization imports
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

%matplotlib inline

In [0]:
def read_the_dat(filepath):
    with urlopen(filepath) as raw_data:
        datfileL=pd.DataFrame(l.decode("utf-8").rstrip().split('::') for l in raw_data)
    return datfileL

In [1]:
movie_fname = 'https://raw.githubusercontent.com/lazysimon/NYCDSA_CapstoneProject/master/latest/movies.dat'
df_movies = read_the_dat(movie_fname)
df_movies.columns = ['movieID', 'title', 'genres']
df_movies['movieID'] = df_movies['movieID'].astype(int)
df_movies.head()

NameError: ignored

In [0]:
rating_fname = 'https://raw.githubusercontent.com/lazysimon/NYCDSA_CapstoneProject/master/latest/ratings.dat'
df_ratings = read_the_dat(rating_fname)
df_ratings.columns = ['userID', 'movieID', 'rating', 'timestamp']
df_ratings['userID'] = df_ratings['userID'].astype(int)
df_ratings['movieID'] = df_ratings['movieID'].astype(int)
df_ratings['rating'] = df_ratings['rating'].astype(int)
df_ratings['timestamp'] = df_ratings['timestamp'].astype(int)
df_ratings.head()

,userID,movieID,rating,timestamp
0,1,111161,10,1373234211
1,1,117060,7,1373415231
2,1,120755,6,1373424360
3,1,317919,6,1373495763
4,1,454876,10,1373621125


In [0]:
len(rating_df.userID.unique())

59168

In [0]:
rating_tmp = pd.DataFrame(rating_df.groupby('rating').size(), columns=['count'])
rating_tmp.hist()

,count
rating,
0,237
1,9533
2,8213
3,13695
4,25105
5,60047
6,105219
7,180688
8,193298
